# HODDIES Tutorial


## 1. Introduction to HODDIES and Halo Occupation Distribution (HOD) Models

HODDIES is a Python package designed to compute Halo Occupation Distribution (HOD) models and generate mock galaxy catalogues. It provides functionalities to link galaxies to dark matter halos based on various HOD formalisms.

### What is the Halo Occupation Distribution (HOD)?

The Halo Occupation Distribution (HOD) is a statistical framework used in cosmology to describe how galaxies populate dark matter halos. It specifies the probability that a dark matter halo of a given mass $M$ contains a certain number of galaxies. The HOD is typically broken down into two components:

1.  **Central Galaxies**: The probability of a halo hosting a central galaxy. Each halo is assumed to host at most one central galaxy.
2.  **Satellite Galaxies**: The mean number of satellite galaxies in a halo, given that it hosts a central galaxy. These satellites are typically distributed within the halo following a certain profile (e.g., NFW profile).

The total mean number of galaxies in a halo of mass $ M $ is given by:

$$\langle N_{\text{gal}}(M) \rangle = \langle N_{\text{cen}}(M) \rangle + \langle N_{\text{sat}}(M) \rangle$$

### HOD Models in HODDIES

The `HODDIES/HODDIES/HOD_models.py` file implements several HOD models. Here's a brief overview of some of them:

#### a) Standard HOD (SHOD) - (Zheng et al. 2007)

This model is commonly used and describes the mean number of central galaxies with a softened step function, and satellite galaxies with a power law.

**Central Galaxy Occupation:**
$$\langle N_{\text{cen}}(M) \rangle = \frac{1}{2} \left[ 1 + \mathrm{erf} \left( \frac{\log_{10} M - \log_{10} M_{\text{min}}}{\sigma_{\log M}} \right) \right]$$
Where:
- $M$: Halo mass
- $M_{\text{min}}$: Characteristic minimum halo mass for a central galaxy
- $\sigma_{\log M}$: Width of the transition in central occupation

**Satellite Galaxy Occupation:**
$$\langle N_{\text{sat}}(M) \rangle = A_s \left( \frac{M - M_0}{M_1} \right)^\alpha$$
Where:
- $M_0$: Mass threshold below which no satellites form
- $M_1$: Characteristic mass scale for hosting one satellite galaxy
- $\alpha$: Power-law slope for satellite occupation
- $A_s$: Normalization amplitude for satellites

The implementation in `HOD_models.py` for `SHOD` (lines 164-186) corresponds to the central occupation, and `Nsat_pow_law` (lines 188-215) corresponds to the satellite occupation.

#### b) Gaussian HOD (GHOD) - (arXiv:1708.07628)

This model uses a Gaussian function to describe the central galaxy occupation.

$$\langle N_{\text{cen}}(M) \rangle = \frac{A_c}{\sqrt{2 \pi} \sigma_M} \exp\left(-\frac{(\log_{10} M - M_c)^2}{2 \sigma_M^2}\right)$$
Where:
- $M_c$: Mean halo mass (log10)
- $\sigma_M$: Width of central galaxy mass distribution
- $A_c$: Normalization amplitude

(See `GHOD` function in `HOD_models.py`, lines 75-99)

#### c) Log-normal HOD (LNHOD) - (arXiv:2306.06319)

The LNHOD model, based on arXiv:2306.06319, uses a log-normal distribution for central galaxy occupation. The function returns 0 if $x \le 0$, otherwise:

$$\langle N_{\text{cen}}(M) \rangle = \frac{A_c}{x \sigma_M \sqrt{2 \pi}} \exp\left(-\frac{(\log(x))^2}{2 \sigma_M^2}\right)$$

Where:
- $M$: Halo mass (represented as `log10_Mh` in the code, and $x$ is derived from it).
- $A_c$: Normalization amplitude (`Ac`).
- $M_c$: Mean halo mass (log10) (`Mc`).
- $\sigma_M$: Width of central galaxy mass distribution (`sig_M`).
- $x = 10^{\log_{10} M} - 10^{M_c} + 1$ (approximated as `log10_Mh - Mc + 1` in the code for the log-transformed mass difference, where `log10_Mh` and `Mc` are already log values).

(See `LNHOD` function in `HOD_models.py`, lines 101-129)

#### d) Star-forming HOD (SFHOD) - (arXiv:1708.07628)

The SFHOD model, based on arXiv:1708.07628, has a piecewise definition for central galaxy occupation:

If $M_c \ge \log_{10} M$:
$$\langle N_{\text{cen}}(M) \rangle = \frac{A_c}{\sqrt{2 \pi} \sigma_M} \exp\left(-\frac{(\log_{10} M - M_c)^2}{2 \sigma_M^2}\right)$$

Else (if $\log_{10} M > M_c$):
$$\langle N_{\text{cen}}(M) \rangle = \frac{A_c}{\sqrt{2 \pi} \sigma_M} \left( \frac{10^{\log_{10} M}}{10^{M_c}} \right)^\gamma$$

Where:
- $M$: Halo mass (represented as `log10_Mh`).
- $A_c$: Normalization amplitude (`Ac`).
- $M_c$: Mean halo mass (log10) (`Mc`).
- $\sigma_M$: Width of central galaxy mass distribution (`sig_M`).
- $\gamma$: Shape parameter controlling the asymmetry (`gamma`).
(See `SFHOD` function in `HOD_models.py`, lines 131-161)

#### e) High Mass Quenched (HMQ) and modified HMQ (mHMQ) - (arXiv:1910.05095, arXiv:2306.06319)

These are more complex models, particularly useful for distinguishing between quenched and star-forming galaxy populations. HMQ includes a quenching factor (Q) and a maximum probability (pmax). mHMQ is a simplified version without quenching terms.

The HMQ model, based on arxiv:1910.05095, describes the expected number of galaxies as:

$$\text{HMQ}(\log_{10} M_h) = A_c \left( 2 A \phi_x \Phi_{\gamma,x} + \frac{0.5}{Q} \left( 1 + \mathrm{erf}\left(\frac{\log_{10} M_h - M_c}{0.01}\right) \right) \right)$$

Where:
- $\log_{10} M_h$: Logarithm (base 10) of halo mass.
- $A_c$: Normalization amplitude (`Ac`).
- $M_c$: Mean halo mass (log10) (`Mc`).
- $\sigma_M$: Width of central galaxy mass distribution (`sig_M`).
- $\gamma$: Shape parameter controlling the asymmetry (`gamma`).
- $Q$: Quenching factor (`Q`).
- $pmax$: Maximum probability (`pmax`).
- $A = (pmax - 1/Q)$.
- $\phi_x = \frac{1}{\sqrt{2 \pi} \sigma_M} \exp\left(-\frac{(\log_{10} M_h - M_c)^2}{2 \sigma_M^2}\right)$.
- $\Phi_{\gamma,x} = 0.5 \left( 1 + \mathrm{erf}\left(\frac{\gamma (\log_{10} M_h - M_c)}{\sigma_M \sqrt{2}}\right) \right).

(See `HMQ` and `mHMQ` functions in `HOD_models.py`, lines 6-41 and 43-73 respectively)

The specific parameters for each HOD model are defined within the functions in `HOD_models.py`, and their interpretation will depend on the chosen model.






## 2. Parameter File and Running the Code

HODDIES uses a YAML-formatted parameter file to configure the HOD model, simulation settings, and analysis parameters. This file acts as a central control for your simulations and analyses. Let's break down the key sections of a typical parameter file.

### Parameter File Structure (e.g., `HODDIES/default_HOD_parameters.yaml`)

A parameter file in HODDIES generally includes the following sections:

#### a) `tracers`

This section lists the galaxy populations (tracers) you want to model. Each tracer will have its own set of HOD parameters defined subsequently. For exemple:

```
tracers:
  - LRG
  - ELG
  - QSO
```

#### b) HOD Model Parameters (e.g., `LRG`, `ELG`, `QSO` sections)

For each tracer defined in the `tracers` section, there will be a corresponding section detailing its HOD parameters. These parameters govern the central and satellite galaxy occupations within halos.

**Central Galaxy Parameters:**
- `HOD_model`: Specifies the HOD model for central galaxies (e.g., `SHOD`, `GHOD`, `HMQ`, `mHMQ`, `LNHOD`, `SFHOD`).
- `Ac`: Normalization amplitude.
- `log_Mcent`: Characteristic minimum halo mass (log10) for central galaxies.
- `sigma_M`: Width of the central galaxy mass distribution or steepness of the step function.
- `gamma`, `Q`, `pmax`: Additional parameters depending on the chosen `HOD_model` (e.g., `HMQ`, `mHMQ`, `SFHOD`).

**Satellite Galaxy Parameters:**
- `satellites`: Boolean, `true` if satellite galaxies are included.
- `sat_HOD_model`: HOD model for satellite galaxies (e.g., `Nsat_pow_law`).
- `As`: Normalization amplitude for satellites.
- `M_0`, `M_1`, `alpha`: Parameters for the satellite power-law model.
- `f_sigv`: Scaling factor for velocity dispersion.
- `vel_sat`: Velocity distribution model for satellites (e.g., `rd_normal`, `NFW`, `infall`).
- `density`: Target galaxy number density.
- `vsmear`: Redshift error smearing. Can be a float for a fixed value or a list `[zmin, zmax]` to use DESI redshift error distribution.

**Other Parameters:**
- `assembly_bias`: Parameters related to assembly bias.
- `conformity_bias`: Boolean, to include conformity bias.
- `exp_frac`, `exp_scale`, `nfw_rescale`: Parameters for modified NFW profiles.

#### c) `hcat` (Halo Catalog and Simulation Settings)

This section configures the halo catalog from which mock galaxies will be generated.

- `boxsize`: Simulation box size.
- `path_to_sim`: Base path to halo catalogs.
- `z_simu`: Redshift of the simulation snapshot.
- `Abacus`: Specific settings for AbacusSummit simulations (e.g., `sim_name`, `load_particles`).
- `Uchuu`: Specific settings for Uchuu simulations (e.g., `sim_name`, `path_to_sim`, `load_subhalos`).

#### d) `2PCF_settings` (Two-Point Correlation Function Settings)

These parameters control how the two-point correlation function (2PCF) is computed.

- `rsd`: Boolean, whether to include redshift-space distortions.
- `bin_logscale`: Boolean, whether to use logarithmic binning for `r`.
- `multipole_index`: List of multipole moments to compute (e.g., `0` for monopole, `2` for quadrupole).
- `rmax`, `rmin`, `rp_max`, `rp_min`: Radial and projected separation ranges.
- `n_r_bins`, `n_rp_bins`, `n_mu_bins`: Number of bins for various correlation function calculations.
- `pimax`: Maximum $\pi$ for $w_p$ integration.

#### e) `cosmo` (Cosmology Settings)

This section allows you to define the cosmology, though it's often automatically set if using AbacusSummit simulations.

- `fiducial_cosmo`: Name of a predefined cosmology (e.g., `Planck2018FullFlatLCDM`).
- `engine`: Cosmology backend (e.g., `class`).
- `Omega_m`, `Omega_cdm`, `Omega_L`, `Omega_b`, `sigma_8`, `n_s`, `w0_fdl`, `wa_fdl`: Cosmological parameters.

#### f) Global Settings

- `seed`: Global random seed for reproducibility.
- `nthreads`: Number of threads to use for parallel processing.
- `use_assembly_bias`, `use_particles`: Global flags.

#### g) `fit_param` (Fitting Settings)

This section is crucial for configuring the fitting and minimization procedures.

- `nb_real`: Number of mock catalogs to generate for each HOD parameter set.
- `fit_name`: Name identifier for this fitting run.
- `path_to_training_point`: null  # Path to save or read precomputed training samples
- `dir_output_fit`: path_to_save_fit_outputs  # Directory where output from this fitting will be saved
- `fit_type`: wp+xi  # Type of 2PCF(s) used in fitting: wp = projected, xi = multipoles
- `generate_training_sample`: True  # Whether to generate a new training sample using sampling_type
- `sampling_type`: Hammersley  # Sampling method for generating initial training points. Choices: LHS or Hammersley 
- `N_trainning_points`: 800  # Number of training points to generate
- `seed_trainning`: 18  # Random seed for reproducibility of LHS sampling
- `n_calls`: 800  # Number of GP iterations (total number of sampling steps)
- `logchi2`: True  # Use log(chi²) instead of raw chi² in GP modeling (recommended)

- `sampler`: 'emcee'  # MCMC sampler to use ('emcee' or 'zeus')
- `n_iter`: 10000  # Number of iterations for MCMC sampling
- `nwalkers`: 20  # Number of walkers for the MCMC sampler
- `func_aq`: 'EI'  # Acquisition function for GP-based optimization ('EI' = Expected Improvement)

- `length_scale_bounds`: [0.001, 10]  # Bounds for GP kernel length scale optimization
- `length_scale`: False  # If set to a list, GP kernel uses fixed length scales
- `kernel_gp`: Matern_52  # Type of GP kernel to use

- `save_fn`: 'results_fit.npy'

- `use_desi_data`: True 
- `zmin`: 0.8
- `zmax`: 1.1
- `dir_data`: /global/homes/a/arocher/users_arocher/Y3/loa-v1/v2/PIP/cosmo_0
- `region`: GCcomb
- `weights_type`: pip_angular_bitwise
- `njack`: 128
- `nran`: 4
- `bin_type`: log
- `load_cov_jk`: False  # Use JK covariance matrix
- `corr_dir`: /global/cfs/cdirs/desi/users/arocher/Y1/2PCF_for_corr/Abcaus_small_boxes/  #root directory to find correlation matrices (to be rescaled by JK variances)
- `nb_mocks`: 1883 # Nb of mocks used to compute the covariance
- `use_vsmear`: True
- `fit_type`: wp+xi # Only wp or xi available at the moment

- `priors`:  
    LRG:
      M_0: [12.5, 13.5]
      M_1: [13, 14.5] 
      alpha: [0.5, 1.5]
      f_sigv: [0.5, 1.5]
      log_Mcent: [12.4, 13.5]
      sigma_M: [0.05, 1]
      


### How to Run the Code

To run HODDIES, you typically import the `HOD` class and initialize it with your parameter file. Here's a basic example:

```python
import yaml
from HODDIES import HOD

# Load your parameter file
params = yaml.load(open('your_parameter_file.yaml'), Loader=yaml.FullLoader)

# Initialize the HOD object
hod_obj = HOD(param_file='your_parameter_file.yaml')

# You can then access and modify parameters directly if needed
# hod_obj.args['QSO']['density'] = 0.0006

# To make a mock catalog (more details in the next section)
# mock_catalog = hod_obj.make_mock_cat()

# To run the minimisation (more details later)
# result = hod_obj.minimize_hod()
```

The `param_file` argument in `HOD()` constructor takes the path to your YAML parameter file. You can also pass a dictionary to the constructor directly if you've loaded it with `yaml.load()`.

Now let's move on to generating mock catalogues and computing correlation functions.


## 3. Loading Halo Catalogues, Creating Mock Catalogues, and Computing Correlation Functions

This section will guide you through the process of loading dark matter halo catalogues, populating them with galaxies using an HOD model to create mock catalogues, and then computing two-point correlation functions (2PCF), specifically projected correlation function ($w_p$) and redshift-space multipoles ($\xi_0, \xi_2$).

### a) Loading Halo Catalogues

HODDIES can interface with different halo catalogues, such as AbacusSummit and Uchuu. The selection and configuration of the halo catalogue are done through the `hcat` section of your parameter file.

Here's how you would typically initialize the `HOD` object with a specified halo catalogue. In this example, we'll use AbacusSummit.

First, make sure to import the necessary libraries.



In [14]:
import yaml
from HODDIES import HOD
import numpy as np
import matplotlib.pyplot as plt


Now, let's set up the HOD object, specifying the path to the Abacus simulation and the parameter file. You can also override parameters directly when initializing the `HOD` object.



In [15]:
# Define the redshift of the simulation
z_sim = 0.95

# Path to your Abacus simulation. Please adjust this path to your local setup.
path_to_abacus = '/global/cfs/cdirs/desi/cosmosim/Abacus'

# Load your parameter file
param_file_path = 'HODDIES/desi/test_fit_param_QSO.yaml'
# args = yaml.load(open(param_file_path), Loader=yaml.FullLoader)

# Initialize the HOD object, overriding halo catalog settings for Abacus
hod_obj = HOD(
    path_to_abacus_sim=path_to_abacus,
    param_file=param_file_path,
    hcat={
        'Abacus': {
            'sim_name': 'AbacusSummit_highbase_c000_ph100',
            'z_simu': z_sim
        }
    }
)

print("HOD object initialized successfully!")
print(f"Loaded Abacus simulation: {hod_obj.args['hcat']['Abacus']['sim_name']} at z={hod_obj.args['hcat']['Abacus']['z_simu']}")


Set number of threads to 32
Load Compaso cat from /global/cfs/cdirs/desi/cosmosim/Abacus/AbacusSummit_highbase_c000_ph100/halos/z0.950 ...
Done took 00:00:27
Compute columns...
Done took  00:00:00
AbacusSummit_highbase_c000_ph100 at 0.95 loaded, took 00:00:28
Initialize Abacus c000 cosmology
HOD object initialized successfully!
Loaded Abacus simulation: AbacusSummit_highbase_c000_ph100 at z=0.95


#### Loading Uchuu Halo Catalogues

To load an Uchuu halo catalogue, you would modify the `hcat` section of your `HOD` object initialization. Here's an example:



In [ ]:
# Define the redshift for Uchuu simulation (example)
z_sim_uchuu = 0.94 # Uchuu snapshot [0.43, 0.49, 0.56, 0.63, 0.70, 0.78, 0.86, 0.94, 1.03, 1.12, 1.22, 1.32, 1.43, 1.65]


# Re-initialize the HOD object for Uchuu, overriding halo catalog settings
# You would typically do this in a separate notebook or clear previous HOD object
hod_obj_uchuu = HOD(
    param_file=param_file_path, # Using the same parameter file for HOD settings
    hcat={'z_simu': z_sim_uchuu,
        'Uchuu': { 
            'sim_name': 'Uchuu2Gpc', # Example Uchuu sim name Choices ['Planck18', 'Planck18_DDE', 'DESIY1_DEE', 'Uchuu2Gpc']  
            'load_subhalos': False # Set to True to use subhalos
        }
    },
    read_Uchuu=True,
    nthreads=128
)

print("HOD object initialized with Uchuu simulation successfully!")
print(f"Loaded Uchuu simulation: {hod_obj_uchuu.args['hcat']['Uchuu']['sim_name']} at z={hod_obj_uchuu.args['hcat']['z_simu']}")


Set number of threads to 32
Load Uchuu Uchuu2Gpc simulation at z=0.940 
Reading ['/pscratch/sd/a/arocher/Uchuu/Uchuu_halo_catalogs/halodir_034/Uchuu_halodir_034_halos_subhalos_z0.940_02.h5', '/pscratch/sd/a/arocher/Uchuu/Uchuu_halo_catalogs/halodir_034/Uchuu_halodir_034_halos_subhalos_z0.940_04.h5', '/pscratch/sd/a/arocher/Uchuu/Uchuu_halo_catalogs/halodir_034/Uchuu_halodir_034_halos_subhalos_z0.940_03.h5', '/pscratch/sd/a/arocher/Uchuu/Uchuu_halo_catalogs/halodir_034/Uchuu_halodir_034_halos_subhalos_z0.940_07.h5', '/pscratch/sd/a/arocher/Uchuu/Uchuu_halo_catalogs/halodir_034/Uchuu_halodir_034_halos_subhalos_z0.940_05.h5', '/pscratch/sd/a/arocher/Uchuu/Uchuu_halo_catalogs/halodir_034/Uchuu_halodir_034_halos_subhalos_z0.940_00.h5', '/pscratch/sd/a/arocher/Uchuu/Uchuu_halo_catalogs/halodir_034/Uchuu_halodir_034_halos_subhalos_z0.940_01.h5', '/pscratch/sd/a/arocher/Uchuu/Uchuu_halo_catalogs/halodir_034/Uchuu_halodir_034_halos_subhalos_z0.940_06.h5', '/pscratch/sd/a/arocher/Uchuu/Uchuu_hal

### b) Creating Mock Catalogues

Once the `HOD` object is initialized with the halo catalogue, you can create a mock galaxy catalogue. The `make_mock_cat()` method populates the halos with central and satellite galaxies according to the HOD model specified in the parameter file.

Before creating the mock catalogue, it's often useful to set the HOD parameters. In this example, we'll use some example HOD parameters. In a real scenario, these might come from a fitting procedure.



In [25]:
# Example HOD parameters (these would typically come from a fit)
# Using parameters from QSO_fits.ipynb for z=0.95
ex_hod_params = {
    'M_0': 11.93, 
    'M_1': 13.29, 
    'alpha': 0.84, 
    'f_sigv': 0.99, 
    'log_Mcent': 12.17, 
    'sigma_M': 0.37,
    'density': 0.0005 # Example density from test_fit_param_QSO.yaml
}

# Update the HOD object's parameters for the QSO tracer
hod_obj_uchuu.args['QSO'].update(ex_hod_params)

# Create the mock catalogue
mock_catalogue = hod_obj_uchuu.make_mock_cat(fix_seed=42) # fix_seed for reproducibility

print("Mock catalogue created successfully!")
print(f"Number of galaxies in mock: {len(mock_catalogue)}")
print(mock_catalogue.columns())



Create mock catalog for ['QSO']
Run HOD for QSO
Set density to 0.0005 gal/Mpc/h
HOD Computed 6.220611095428467
Start satellite assignement
Satellite assignement done 0.953514814376831
QSO mock catalogue done 1.630145788192749
3250104 central galaxies, 746921 satellites, fraction of satellite 0.19 
Done overall time  QSO 12.784969329833984
Mock catalogue created successfully!
Number of galaxies in mock: 3997025
['Mh', 'Rh', 'Rs', 'Vrms', 'c', 'halo_id', 'log10_Mh', 'vx', 'vy', 'vz', 'x', 'y', 'z', 'Central', 'TRACER']


The `mock_catalogue` object is a table containing the positions, velocities, and other properties of the mock galaxies. You can inspect its columns as shown above.

### c) Computing Correlation Functions (wp and xi)

HODDIES can compute various two-point correlation functions from your mock catalogues. The `compute_observable()` method is used for this purpose.

We will compute the projected correlation function ($w_p$) and the redshift-space multipoles (monopole $\xi_0$ and quadrupole $\xi_2$). The settings for these computations are defined in the `2PCF_settings` section of your parameter file. We will use the default settings from the loaded `test_fit_param_QSO.yaml`.



In [ ]:
# Compute the correlation functions
# The fit_type is set to 'wp+xi' in test_fit_param_QSO.yaml, so it will compute both.
computed_cf = hod_obj.get_wp(mock_catalogue, tracer_name='QSO')

print("Correlation functions computed successfully!")
print("Keys available in computed_cf:", computed_cf.keys())



The `computed_cf` dictionary will contain the results, typically `'wp'` for projected correlation function and `'xi'` for redshift-space multipoles. The `'xi'` entry will itself be a dictionary containing `'xi0'` and `'xi2'`.

### d) Plotting the Outputs

Finally, let's visualize the computed correlation functions.



In [ ]:
# Extract results for plotting
rp_bins = computed_cf['wp']['r'] # Radial bins for wp
wp = computed_cf['wp']['wprp']

s_bins = computed_cf['xi']['r'] # Radial bins for xi multipoles
xi0 = computed_cf['xi']['xi0']
xi2 = computed_cf['xi']['xi2']

# Plot wp
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.errorbar(rp_bins, wp, fmt='o-', label='$w_p(r_p)$')
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$r_p$ (Mpc/h)')
plt.ylabel(r'$w_p(r_p)$')
plt.title('Projected Correlation Function')
plt.legend()
plt.grid(True)

# Plot xi multipoles
plt.subplot(1, 2, 2)
plt.errorbar(s_bins, xi0, fmt='o-', label=r'$\xi_0(s)$')
plt.errorbar(s_bins, xi2, fmt='s-', label=r'$\xi_2(s)$')
plt.xscale('log')
plt.xlabel(r'$s$ (Mpc/h)')
plt.ylabel(r'$\xi(s)$')
plt.title('Redshift-Space Correlation Function Multipoles')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()



## 4. Minimization Procedure and Results

HODDIES provides functionality to perform a minimization procedure to find the best-fit HOD parameters by comparing the mock galaxy correlation functions with observed data. This typically involves an optimization algorithm that iteratively adjusts HOD parameters to minimize a $\chi^2$ value.

### a) Describing the Minimization Procedure

The minimization in HODDIES often leverages techniques like Gaussian Process Regression (GPR) coupled with Markov Chain Monte Carlo (MCMC) sampling. The general idea is:

1.  **Define Priors**: Specify the allowed ranges for each HOD parameter in your parameter file under the `fit_param -> priors` section.
2.  **Generate Training Samples (GPR)**: If `generate_training_sample` is `True` in `fit_param`, the code will generate a set of HOD parameter combinations (training points) within the defined priors. For each training point, it generates mock catalogues and computes their correlation functions. This information is used to train a Gaussian Process (GP) model, which acts as a fast emulator of the HOD model.
3.  **Minimization/Sampling (MCMC)**: An MCMC sampler (e.g., `emcee` or `zeus`) is then used to explore the parameter space, guided by the GP model. The sampler aims to find the set of HOD parameters that best fit the observational data by minimizing a $\chi^2$ likelihood. The `func_aq` (acquisition function) parameter in `fit_param` determines how new sampling points are chosen to improve the GP model.

Key parameters for the minimization are set in the `fit_param` section of your YAML file, including `nb_real`, `fit_name`, `dir_output_fit`, `fit_type`, `sampling_type`, `N_trainning_points`, `n_calls`, `sampler`, `n_iter`, `nwalkers`, and `priors`.

### b) How to Run the Minimization

To run the minimization, you use the `minimize_hod()` method of the `HOD` object. Before running, ensure your parameter file is correctly configured with the `fit_param` section, including appropriate priors and settings for the minimizer and sampler.

Let's re-initialize the HOD object (if not already done) and then run a simplified minimization example. For a real fit, `maxiter` and `n_calls` would be significantly larger.



In [ ]:
# Re-initialize HOD object (if running this cell independently)
# z_sim = 0.95 # From previous section
# path_to_abacus = '/global/cfs/cdirs/desi/cosmosim/Abacus' # From previous section
# param_file_path = 'HODDIES/desi/test_fit_param_QSO.yaml' # From previous section
# hod_obj = HOD(
#     path_to_abacus_sim=path_to_abacus,
#     param_file=param_file_path,
#     hcat={
#         'Abacus': {
#             'sim_name': 'AbacusSummit_highbase_c000_ph100',
#             'z_simu': z_sim
#         }
#     }
# )

print("Running a simplified minimization (this might take some time for real runs)...")

# Run the minimizer
# We set a very small number of iterations for demonstration purposes.
# For a real analysis, n_calls and n_iter (if using MCMC) would be much larger.
# The save_fn parameter specifies the name of the file where the fit results will be saved.
result_minimization = hod_obj.minimize_hod(
    minimizer_options={
        "maxiter": 2, # Very small for example, increase for real runs
        "popsize": 2, 
        'xtol':1e-6
    },
    save_fn='tutorial_fit_result',
    load_cov_jk=False, # Use jackknife covariance if True, otherwise assumes precomputed cov.
    add_poisson_noise=False # Add Poisson noise to mocks
)

print("Minimization completed!")
print("Best-fit parameters (raw output):")
print(result_minimization['x'])



The `result_minimization` object (or the saved `.npy` file) contains the best-fit parameters and other information from the minimization. The `x` key typically holds the array of best-fit HOD parameters.

To see the best-fit parameters in a more readable dictionary format, you can map them back to their names using the `priors` keys:



In [ ]:
# Get the names of the parameters from the priors
param_names = hod_obj.args['fit_param']['priors']['QSO'].keys()

# Map the best-fit values to their parameter names
best_fit_params_dict = dict(zip(param_names, result_minimization['x']))

print("Best-fit HOD Parameters:")
for param, value in best_fit_params_dict.items():
    print(f"{param}: {value:.4f}")



### c) Plotting the Results

HODDIES provides a convenient method, `plot_bf_data()`, to visualize the best-fit HOD model's correlation functions against the observed data. This function takes the path to the best-fit file (saved by `minimize_hod`) and can plot the mocks generated with the best-fit parameters.



In [ ]:
# Plot the best-fit results
# The bf_file parameter should point to the .npy file saved by minimize_hod
# In this example, it's 'tutorial_fit_result.npy'
fig = hod_obj.plot_bf_data(
    bf_file='tutorial_fit_result.npy',
    fix_seed=10, # Use a fixed seed for mock generation for consistent plotting
    add_no_vsmear=True, # Option to plot with and without velocity smearing
    verbose=True # Print more details during plotting
)

plt.show()

print("Tutorial complete! You have learned how to initialize HODDIES, create mock catalogues, compute correlation functions, and perform a basic minimization.")

